In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('train.csv', index_col=0)
# test = pd.read_csv('test.csv')
# test_labels = pd.read_csv('test_labels.csv')

# split the data in train.csv into training data and testing data
X_train, X_test, y_train, y_test = train_test_split(
     train.iloc[:, 0], train.iloc[:, 1:], test_size=0.2, random_state=123)

In [3]:
# use binary counts as features
vectorizer = CountVectorizer(binary=True)
X_train_counts = vectorizer.fit_transform(X_train)

In [4]:
# use OneVsRestClassifier with Multinomial Naive Bayes since it's a multi-label classification problem
clf = OneVsRestClassifier(MultinomialNB()).fit(X_train_counts, y_train)

In [5]:
X_test_counts = vectorizer.transform(X_test)
#y_pred = clf.predict_proba(X_test_counts)

In [6]:
clf.score(X_test_counts, y_test)

0.9083189722700924

In [7]:
# TODO: who does this not work?
# roc_auc_score(y_test, y_pred, average='samples')